In [46]:
import requests
from bs4 import BeautifulSoup

#load the webpage
r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c=r.content

#create parsed object using BeautifulSoup
soup = BeautifulSoup(c, "html.parser")

#parse using find_all to get divs with class property row .... aka load the listings on the page
all = soup.find_all("div", {"class","propertyRow"})

#look for price of first unit on the page return just the price as a text
all[0].find("h4", {"class","propPrice"}).text.replace("\n", "").replace(" ", "")

'$725,000'

In [55]:
#for loop through each item on the page to return all of the prices
for item in all:
    print(item.find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", ""))
    #now add two lines to return the street address on one line and the city/state info on another
    print(item.find_all("span", {"class" : "propAddressCollapse"})[0].text)
    print(item.find_all("span", {"class" : "propAddressCollapse"})[1].text)
    #now extract number of beds/bath
    #try to pass bed info, but if nothing there then pass
    try:
        print(item.find("span", {"class" , "infoBed"}).find("b").text + " bed")
    except:
        print(None)
    #try to pass bathroom info, but if nothing there then pass
    try:
        print(item.find("span", {"class" , "infoSqFt"}).find("b").text + " SqFt")
    except:
        print(None)
    #try to pass square footage, but if nothing there then pass
    try:
        print(item.find("span", {"class" , "infoValueFullBath"}).find("b").text + " full bath")
    except:
        print(None)
    #try to pass half bath, but if nothing there then pass
    try:
        print(item.find("span", {"class" , "infoValueHalfBath"}).find("b").text + " half bath")
    except:
        print(None)
    print(" ")

$725,000
0 Gateway
Rock Springs, WY 82901
None
None
None
None
 
$452,900
1003 Winchester Blvd.
Rock Springs, WY 82901
4 bed
None
4 full bath
None
 
$396,900
600 Talladega
Rock Springs, WY 82901
5 bed
3,154 SqFt
3 full bath
None
 
$389,900
3239 Spearhead Way
Rock Springs, WY 82901
4 bed
3,076 SqFt
3 full bath
1 half bath
 
$254,000
522 Emerald Street
Rock Springs, WY 82901
3 bed
1,172 SqFt
3 full bath
None
 
$252,900
1302 Veteran's Drive
Rock Springs, WY 82901
4 bed
1,932 SqFt
2 full bath
None
 
$210,000
1021 Cypress Cir
Rock Springs, WY 82901
4 bed
1,676 SqFt
3 full bath
None
 
$209,000
913 Madison Dr
Rock Springs, WY 82901
3 bed
1,344 SqFt
2 full bath
None
 
$199,900
1344 Teton Street
Rock Springs, WY 82901
3 bed
1,920 SqFt
2 full bath
None
 
$196,900
4 Minnies Lane
Rock Springs, WY 82901
3 bed
1,664 SqFt
2 full bath
None
 
